Imports

In [184]:
import numpy as np

todo: initialise constants from the dataset (drop name column)
todo: redefine feed forward generally
todo: do todo 

Intialise network architecture

In [185]:
L = 3  # number of layers(excluding input layer)
n = 2  # number of features
m = 1  # number of output nodes

hidden_layers = 3  # number of layers(excluding input layer)
features = 2  # number of features
outputs = 1  # number of output nodes

In [186]:
def init_network(input_size=features, hidden_size=features+1, output_size=outputs, num_hidden_layers=hidden_layers):
    weights = []
    biases = []
    
    alpha = 1  # scaling factor for weights
    beta = 1   # scaling factor for biases

    # Input to first hidden layer
    weights.append(np.random.randn(hidden_size, input_size) * alpha)
    biases.append(np.random.randn(hidden_size, 1) * beta)
    
    # Hidden layers
    for i in range(num_hidden_layers - 1):
        weights.append(np.random.randn(hidden_size, hidden_size) * alpha)
        biases.append(np.random.randn(hidden_size, 1) * beta)
    
    # Last hidden to output layer
    weights.append(np.random.randn(output_size, hidden_size) * alpha)
    biases.append(np.random.randn(output_size, 1) * beta)
    
    return weights, biases

In [187]:
weights, biases = init_network()
print("Weights of the network:", weights)
print("Biases of the network:", biases)

Weights of the network: [array([[-0.67067813,  1.58424556],
       [-1.56716802, -0.13680524],
       [ 0.363363  , -1.34396603]]), array([[ 0.64717752,  0.09435585,  1.11632814],
       [ 0.09918029, -0.26268643,  1.66045796],
       [ 0.53110829,  0.71613359, -0.35120126]]), array([[-0.07381484,  0.76504959,  1.01929505],
       [-1.23256697, -1.38050319,  1.55976749],
       [ 0.41536909, -0.76064943, -0.91258583]]), array([[ 1.09904145, -0.50794855,  0.21683331]])]
Biases of the network: [array([[ 0.92908401],
       [ 0.11852676],
       [-1.07764212]]), array([[-1.25924555],
       [ 0.08180502],
       [ 1.13123935]]), array([[ 0.44125189],
       [-1.28014511],
       [ 1.71620452]]), array([[2.40131438]])]


Training data

In [188]:
def prepare_data():
  X = np.array([
      [150, 70],
      [254, 73],
      [312, 68],
      [120, 60],
      [154, 61],
      [212, 65],
      [216, 67],
      [145, 67],
      [184, 64],
      [130, 69]
  ])
  y = np.array([0,1,1,0,0,1,1,0,1,0])
  m = y.shape[0]  # number of samples
  # Transpose the input matrix to match the expected shape
  A0 = X.T
  # Reshape y to a matrix
  Y = y.reshape(outputs, m)

  return A0, Y

In [189]:
x_train, y_train = prepare_data()

Activation function sigmoid 
$$g(z)=\frac{1}{(1+e^{-z})}$$

In [190]:
# Sigmoid activation function
def sigmoid(arr):
    return 1 / (1+np.exp(-1*arr))

Feed forward

In [191]:
def feed_forward(input_layer=x_train, num_hidden_layers=hidden_layers, weights=weights, biases=biases):
  cache = [input_layer]
  
  for i in range(0,num_hidden_layers+1):
    prev_layer = cache[i]
    W = weights[i]
    b = biases[i]
    Z = W @ prev_layer + b
    A = sigmoid(Z)
    cache.append(A)

  return cache


In [192]:
cache = feed_forward(x_train)  # Get the output layer predictions
predictions = cache[-1]  # Last layer output
print("Predictions of the network:", cache[-1])
print("\nPredictions:", cache)

Predictions of the network: [[0.96614655 0.96555063 0.96824829 0.96614656 0.96554848 0.96554679
  0.96554676 0.96614653 0.96554674 0.96614656]]

Predictions: [array([[150, 254, 312, 120, 154, 212, 216, 145, 184, 130],
       [ 70,  73,  68,  60,  61,  65,  67,  67,  64,  69]]), array([[9.99986658e-001, 4.43181047e-024, 2.05433234e-044,
        9.99999815e-001, 3.28084458e-003, 2.37555084e-017,
        3.86131563e-017, 9.99945932e-001, 6.97181552e-010,
        1.00000000e+000],
       [6.31900549e-107, 6.89787712e-178, 4.57365844e-217,
        6.50367817e-086, 4.10131284e-109, 7.93866207e-149,
        1.14411611e-151, 2.40963983e-103, 1.03826669e-129,
        2.96695497e-093],
       [2.21603168e-018, 1.01387704e-003, 9.99999999e-001,
        2.80640176e-017, 1.69783481e-012, 1.11676611e-005,
        3.24965902e-006, 2.03036594e-017, 1.63328048e-009,
        5.93091968e-021]]), array([[0.35158563, 0.22129877, 0.46433134, 0.35158757, 0.22146968,
        0.22110594, 0.22110442, 0.35157962

Use loss as cost function: $L\left(\hat{y}_i y_i\right)=-\left(y_i \ln \hat{y}_i+\left(1-y_i\right) \ln \left(1-\hat{y}_i\right)\right)$


Hence cost given by: $C=\frac{1}{m} \sum_{i=1}^m L\left(\hat{y}_i, y_i\right)$

In [193]:
def cost(y_hat=predictions, y=y_train):
  """
  y_hat should be a n^L x m matrix
  y should be a n^L x m matrix
  """

  # Clip predictions to avoid log(0)
  y_hat = np.clip(y_hat, 1e-15, 1 - 1e-15)

  # 1. losses is a n^L x m
  losses = - ( (y * np.log(y_hat)) + (1 - y)*np.log(1 - y_hat) )

  return np.mean(losses)

In [194]:
print(cost(predictions, y_train))

1.70835654655431


Backpropagation with formulas derived using calculus

In [195]:
def backprop_layer(dC_dA_current, A_current, A_prev, W_current, Y, layer_num):
    """
    Backpropagation for a single layer.

    Args:
        dC_dA_current:  Propagator from the next layer (dC/dA[l+1]). For the output layer, this is None.
        A_current:      Activations of the current layer (A[l])
        A_prev:         Activations of the previous layer (A[l-1])
        W_current:      Weights of the current layer (W[l])
        Y:              True labels (only needed for the output layer)

    Returns:
        dC_dW: Gradient of cost with respect to weights
        dC_db: Gradient of cost with respect to biases
        dC_dA_prev: Propagator to the previous layer (dC/dA[l-1])
    """

    # Calculate dC/dZ
    if dC_dA_current is None:  # Output layer
        dC_dZ = (1/m) * (A_current - Y)
    else:  # Hidden layer
        dA_dZ = A_current * (1 - A_current)
        dC_dZ = dC_dA_current * dA_dZ


    # calculate dC/dW
    dZ_dW = A_prev
    dC_dW = dC_dZ @ dZ_dW.T

    # step 3. calculate dC/db
    dC_db = np.sum(dC_dZ, axis=1, keepdims=True)

    # calculate propagator to the prev layer
    if layer_num != 0:  # Not the input layer
        dZ_dA_prev = W_current
        dC_dA_prev = W_current.T @ dC_dZ

    else:
        dC_dA_prev = None

    return dC_dW, dC_db, dC_dA_prev

Train

In [196]:
def train(epochs=200, alpha=0.1, hidden_layers=hidden_layers, features=features, outputs=outputs):
  global weights, biases, x_train, y_train
  
  costs = []
  num_layers = hidden_layers + 1

  for e in range(epochs):
    # Feed forward
    cache = feed_forward(x_train)
    predictions = cache[-1]
    
    # Cost
    costs.append(cost(predictions, y_train))

    # Backpropagation 
    dC_dA_prev = None
    updates = []
    for layer_num in range(num_layers, 0, -1):
        
        A_current = cache[layer_num]
        A_prev = cache[layer_num-1]
        W_current = weights[layer_num-1]

        dC_dW, dC_db, dC_dA_prev_new = backprop_layer(
            dC_dA_current=dC_dA_prev, 
            A_current=A_current, 
            A_prev=A_prev, 
            W_current=W_current, 
            Y=y_train,
            layer_num=layer_num
        )
        dC_dA_prev = dC_dA_prev_new
        updates.insert(0, (dC_dW, dC_db))


    # Update weights and biases
    for layer_num in range(num_layers):
        dC_dW, dC_db = updates[layer_num]

        # Update weights and biases
        weights[layer_num] -= alpha * dC_dW
        biases[layer_num] -= alpha * dC_db


    if e % 20 == 0:
      print(f"epoch {e}: cost = {costs[e]:4f}")
  
  return costs

In [197]:
for i in range(10,1,-1):
  print(i)

10
9
8
7
6
5
4
3
2


In [198]:
costs = train()

epoch 0: cost = 1.708357
epoch 20: cost = 0.689665
epoch 40: cost = 0.675130
epoch 60: cost = 0.615251
epoch 80: cost = 0.415680
epoch 100: cost = 0.211029
epoch 120: cost = 0.122125
epoch 140: cost = 0.082644
epoch 160: cost = 0.061585
epoch 180: cost = 0.048745
